<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Notebook 2: TESTING Ames Housing Data
### Presented by:  Stephen Strawbridge

### Purpose of this notebook is to clean/modify the testing dataset to be later used in modeling notebook.



---
## Imports & Data Cleaning

In [1]:
#Relevant imports for this project, including scikit learn modeling imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge

### *Testing Data Cleaning*

In [2]:
#Read in training dataset 
ames_test = pd.read_csv('./datasets/test.csv')

In [3]:
#Check layout of data using head()
ames_test.head(3)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New


In [4]:
#Look at general info of dataset
ames_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

In [5]:
#Change MS_Subclass to an object, as even though these values are numbers, they will be dummified as categories later on in project
ames_test['MS SubClass'] = ames_test['MS SubClass'].astype(str)

In [6]:
#Display only columns with null values (considering the many number of columns)
null_columns = ames_test.columns[ames_test.isnull().any()]
ames_test[null_columns].isnull().sum()

#Source: https://dzone.com/articles/pandas-find-rows-where-columnfield-is-null

Lot Frontage      160
Alley             820
Mas Vnr Type        1
Mas Vnr Area        1
Bsmt Qual          25
Bsmt Cond          25
Bsmt Exposure      25
BsmtFin Type 1     25
BsmtFin Type 2     25
Electrical          1
Fireplace Qu      422
Garage Type        44
Garage Yr Blt      45
Garage Finish      45
Garage Qual        45
Garage Cond        45
Pool QC           874
Fence             706
Misc Feature      837
dtype: int64

In [7]:
#Replace all nulls in object-type columns with "None" using a function

#Define function that iterates through each column and replaces nulls with string 'None'
def replace_nulls(object_columns):
    for column in object_columns:
        ames_test[column].replace(to_replace = np.nan, value = 'None', inplace=True)
    return ames_test

In [8]:
#Define columns that contain objects
object_columns = ['Alley', 'Mas Vnr Type', 'Bsmt Qual', 'Bsmt Exposure', 'Bsmt Cond', 'BsmtFin Type 1', 
                 'BsmtFin Type 2', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Pool QC',
                 'Garage Cond', 'Fireplace Qu', 'Fence', 'Misc Feature', 'Electrical']

#Call the function on the dataset, for columns that contain object datatypes
replace_nulls(object_columns)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,None,None,None,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,None,IR1,Lvl,...,0,0,0,None,None,None,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,None,IR1,Lvl,...,0,0,0,None,None,None,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,None,Reg,Lvl,...,0,0,0,None,None,None,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,None,IR1,Lvl,...,0,185,0,None,None,None,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,None,Reg,Lvl,...,0,0,0,None,None,None,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,None,Reg,Lvl,...,0,0,0,None,MnPrv,None,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,None,Reg,Lvl,...,0,0,0,None,None,None,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,None,Reg,Lvl,...,0,0,0,None,GdWo,None,0,5,2007,WD


In [9]:
#Remaining columns that have null values are now numeric columns
#After reading through data-dictionary, the best strategy arrived at was replacing null values with the average value for the column
ames_test.fillna(ames_test.mean(), inplace=True)

In [10]:
#Double check all nulls are gone
null_columns = ames_test.columns[ames_test.isnull().any()]
ames_test[null_columns].isnull().sum();


In [11]:
#Drop Miscellaneous feature and values columns, as these columns are mostly comprised of null values or zero values
#ames_test.drop(columns=['Misc Feature', 'Misc Val'], inplace=True)

In [12]:
#Lastly, strip spaces and lowercase column names for easier reference
ames_test.columns = ames_test.columns.str.lower().str.replace(' ', '_')

### At this point, TESTING DATAFRAME IS CLEAN

#### Use data dictionaries to dummify categorical/ordinal features into features with numerical values

In [13]:
#Turn 'yes or no' categorial features into 1's and 0's, using map function
ames_test['central_air'] = ames_test['central_air'].map({'Y':1, 'N':0}) 
ames_test['paved_drive'] = ames_test['paved_drive'].map({'Y':1, 'P':0.5, 'N':0}) 

In [14]:
#Turn ordinal/ranked features into numerical rankings ('poor to excellent scale'), using map function
#Note, after submission of this project, I will turn this cell into a function to more neatly organize this notebook

ames_test['bsmt_qual'] = ames_test['bsmt_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})
ames_test['bsmt_cond'] = ames_test['bsmt_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})
ames_test['bsmt_exposure'] = ames_test['bsmt_exposure'].map({'Gd':5, 'Av':3, 'Mn':2, 'No':1, 'NA':0, 'None':0})

ames_test['exter_qual'] = ames_test['exter_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})
ames_test['exter_cond'] = ames_test['exter_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})

ames_test['kitchen_qual'] = ames_test['kitchen_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})
ames_test['fireplace_qu'] = ames_test['fireplace_qu'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})

ames_test['garage_qual'] = ames_test['garage_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})
ames_test['garage_cond'] = ames_test['garage_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})

ames_test['pool_qc'] = ames_test['pool_qc'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0, 'None':0})


In [15]:
#For 'Street' column, turn Pave values into 0 and Grvl values into 1
ames_test['street'] = ames_test['street'].map({'Pave':0, 'Grvl':1}) 

In [16]:
#Double check all nulls are gone
null_columns = ames_test.columns[ames_test.isnull().any()]
ames_test[null_columns].isnull().sum();

### Feature Engineering of Columns

In [17]:
#Engineer a general quality/condition feature
ames_test['general_qual_cond'] = ames_test['overall_qual'] + ames_test['exter_qual']

In [18]:
#Engineer a basement quality/condition feature
ames_test['bsmt_qual_cond'] = ames_test['bsmt_qual'] + ames_test['bsmt_cond'] + ames_test['bsmt_exposure']

In [19]:
#Engineer other qualities feature
ames_test['other_qual'] = ames_test['kitchen_qual'] + ames_test['fireplace_qu']

In [20]:
#Engineer total square footage
ames_test['total_sq_footage'] = ames_test['total_bsmt_sf'] + ames_test['1st_flr_sf'] + ames_test['2nd_flr_sf'] + ames_test['low_qual_fin_sf']

In [21]:
#Engineer total area
ames_test['total_area'] = ames_test['gr_liv_area'] + ames_test['mas_vnr_area'] + ames_test['garage_area'] 

In [22]:
#Engineer 'how_new' feature
ames_test['how_new'] = ames_test['year_built'] + ames_test['year_remod/add']

In [23]:
#Engineer 'total_bath' feature
ames_test['total_bath'] = ames_test['full_bath'] + (0.5 * ames_test['half_bath']) + ames_test['bsmt_full_bath'] + (0.5 * ames_test['bsmt_half_bath'])

In [24]:
#Double check all nulls are gone
null_columns = ames_test.columns[ames_test.isnull().any()]
ames_test[null_columns].isnull().sum()

Series([], dtype: float64)

### Dummify most relevant *nominal* columns/features

In [25]:
#Dummify all desired features
dummy_test = pd.get_dummies(ames_test[['ms_subclass', 'ms_zoning', 'neighborhood', 'bldg_type', 'house_style', 'foundation', 'heating', 'garage_type']],
                             drop_first = True)

### Join dummy dataframe with desired features in testing dataframe, to be used for modeling

In [26]:
#First, isolate only desired columns in the training dataframe
to_combine = ames_test[
    ['general_qual_cond', 'bsmt_qual_cond', 'other_qual', 'total_sq_footage', 'total_area', 'how_new', 'total_bath', #engineered features
     'lot_area', 'garage_cars', 'fireplaces', 'totrms_abvgrd', 'id']] #other desired features

In [27]:
#Then, combine modified training dataframe above with dummy dataframe (we can append, since the order of the 2051 rows has not changed)
combined_ames_test = pd.concat([dummy_test, to_combine], axis=1)

In [28]:
combined_ames_test.shape

(878, 81)

### Finally, find and drop all columns in training set that don't match columns in testing set, and vice versa

In [29]:
#First bring in training dataframe into this notebook
cleaned_modified_train = pd.read_csv('./final_train_test_csv_files/cleaned_modified_train.csv')

In [30]:
#Find columns to drop in TRAINING set (source code leveraged from fellow classmate Zachary Brown)

col_to_drop = []

for col in cleaned_modified_train.columns:
    if col == 'saleprice' or col == 'id' :
        continue
    elif col not in combined_ames_test.columns:
        col_to_drop.append(col)


cleaned_modified_train.drop(columns=col_to_drop, inplace=True)

#Now, all columns in training set are present in testing set (except for 'saleprice' and 'id')

In [31]:
cleaned_modified_train.columns.value_counts()

house_style_SFoyer     1
bsmt_qual_cond         1
ms_subclass_20         1
foundation_PConc       1
neighborhood_Greens    1
                      ..
ms_zoning_RL           1
house_style_2.5Unf     1
garage_cars            1
fireplaces             1
neighborhood_NAmes     1
Length: 81, dtype: int64

In [32]:
#Find columns to drop in TESTING set (source code leveraged from fellow classmate Zachary Brown)

col_to_drop = []

for col in combined_ames_test.columns:
    if col not in cleaned_modified_train.columns:
        col_to_drop.append(col)


combined_ames_test.drop(columns=col_to_drop, inplace=True)

#Now, all columns in testing set are present in training set

In [33]:
combined_ames_test.columns.value_counts()

house_style_SFoyer     1
heating_Grav           1
ms_subclass_20         1
foundation_PConc       1
neighborhood_Greens    1
                      ..
house_style_2.5Unf     1
garage_cars            1
fireplaces             1
house_style_SLvl       1
neighborhood_NAmes     1
Length: 80, dtype: int64

### Save final training and testing set

In [34]:
#Save training data as 'final_ames_train'
cleaned_modified_train.to_csv('./final_train_test_csv_files/final_ames_train.csv', index=False)

#Save testing data as 'final_ames_test'
combined_ames_test.to_csv('./final_train_test_csv_files/final_ames_test.csv', index=False)